In [ ]:
import Pkg; Pkg.add("JLD2")

   Resolving package versions...
      Compat entries added for JLD2
    Updating `~/Macro/Project.toml`
  [033835bb] + JLD2 v0.5.11
    Updating `~/Macro/Manifest.toml`
  [5789e2e9] + FileIO v1.16.6
  [033835bb] + JLD2 v0.5.11
Precompiling project...
   2060.1 ms  ✓ FileIO → HTTPExt
  27171.8 ms  ✓ JLD2
  53618.3 ms  ✓ Latexify → DataFramesExt
  70746.4 ms  ✓ HiGHS
  57780.5 ms  ✓ Plots → UnitfulExt
FileIOExt Waiting for background task / IO / timer.
[pid 3183597] waiting for IO to finish:
 Handle type        uv_handle_t->data
 fs_event           0x2020f20->0x14c126a63ac0
 timer              0x2272be0->0x14c126a63af0
This means that a package has started a background task or event source that has not finished running. For precompilation to complete successfully, the event source needs to be closed explicitly. See the developer documentation on fixing precompilation hangs for more help.
  71797.5 ms  ✓ Plots → FileIOExt
  6 dependencies successfully precompiled in 128 seconds. 238 alre

In [4]:
import Pkg; Pkg.add("FileIO")

   Resolving package versions...
      Compat entries added for FileIO
    Updating `~/Macro/Project.toml`
  [5789e2e9] + FileIO v1.16.6
  No Changes to `~/Macro/Manifest.toml`


In [6]:
using Pkg
#using Revise
Pkg.activate(dirname(dirname(@__DIR__)))
using MacroEnergy
using Gurobi
using Plots
using DataFrames, CSV
using BenchmarkTools
#using JLD2, FileIO
using JuMP

  Activating project at `~/Macro`


In [2]:
VERSION

v"1.11.1"

In [17]:
# Function to get sorting key: returns the index of the first matching suffix or a large number
function suffix_sort_key(name)
    idx = findfirst(s -> endswith(name, s), suffixes)
    return isnothing(idx) ? length(suffixes) + 1 : idx  # Default to placing non-matching names at the end
end

# Function to get flows for each type of asset
function get_flows(asset::Macro.CementPlant, df::DataFrame)
    df[:, "trad_" * string(asset.id) * "_cement"] = Macro.value.(Macro.flow(asset.cement_edge)).data
    df[:, "trad_" * string(asset.id) * "_co2"] = Macro.value.(Macro.flow(asset.co2_edge)).data
end

function get_flows(asset::Macro.ElectrochemCementPlant, df::DataFrame)
    df[:, "echem_" * string(asset.id) * "_cement"] = Macro.value.(Macro.flow(asset.cement_edge)).data
end

function get_flows(asset::ElectricDAC, df::DataFrame)
    df[:, string(asset.id) * "_co2"] = Macro.value.(Macro.flow(asset.co2_edge)).data
end

function get_flows(asset::PowerLine, df::DataFrame)
    df[:, asset.id] = Macro.value.(Macro.flow(asset.elec_edge)).data
end

function get_flows(asset::Battery, df::DataFrame)
    df[:, string(asset.id) * "_charge"] = -1 * Macro.value.(Macro.flow(asset.charge_edge)).data
    df[:, string(asset.id) * "_discharge"] = Macro.value.(Macro.flow(asset.discharge_edge)).data
end

function get_flows(asset::ThermalPower, df::DataFrame)
    df[:, asset.id] = Macro.value.(Macro.flow(asset.elec_edge)).data
    df[:, string(asset.id) * "_co2"] = Macro.value.(Macro.flow(asset.co2_edge)).data
end

function get_flows(asset::VRE, df::DataFrame)
    df[:, asset.id] = Macro.value.(Macro.flow(asset.edge)).data
end

get_flows (generic function with 7 methods)

In [36]:
# Run model for different CO2 caps

no_cap_co2_level = 2.277e7
co2_levels = [0, 0.2, 0.4, 0.6, 0.8]

for co2_level in co2_levels

    # Case name
    case = string(floor(Int, co2_level * 100)) * "pct_co2_reduction"

    # Load system
    case_path = @__DIR__
    println("###### ###### ######")
    println("Running case at $(case_path)")
    system = Macro.load_system(case_path)

    # Set CO2 policy
    co2_policy = no_cap_co2_level * co2_level
    co2_node = Macro.get_nodes_sametype(system.locations, CO2)[1] # There is only 1 CO2 node
    co2_node.rhs_policy[CO2CapConstraint] = co2_policy
    
    # Run model
    model = Macro.generate_model(system)
    Macro.set_optimizer(model, Gurobi.Optimizer);
    Macro.set_optimizer_attributes(model, "BarConvTol"=>1e-3,"Crossover" => 0, "Method" => 2)
    Macro.optimize!(model)

    # Save results
    ## System results
    co2_captured_node = Macro.get_nodes_sametype(system.locations, CO2Captured)[1]
    system_results_df = DataFrame(
        objective_value = Macro.objective_value(model),
        co2_emissions = Macro.value(sum(co2_captured_node.operation_expr[:exogenous])),
        co2_captured = Macro.value(sum(co2_node.operation_expr[:emissions])),
    )
    results_dir = joinpath(case_path, "results")
    mkpath(results_dir)
    Macro.write_csv(joinpath(results_dir, case * "_system_results.csv"), system_results_df)

    ## Capacity results
    capacity_results_df = Macro.get_optimal_asset_capacity(system)
    Macro.write_csv(joinpath(results_dir, case * "_capacity.csv"), capacity_results_df)

    ## 8760 results
    results_8760_df = DataFrame()
        # Get flows of each asset
    for i in eachindex(system.assets)
        get_flows(system.assets[i], results_8760_df)
    end

        # Function to get sorting key: returns the index of the first matching suffix or a large number
    suffixes = ["cement", "co2"] # Define suffixes to reorder dataframe by
    function suffix_sort_key(name)
        idx = findfirst(s -> endswith(name, s), suffixes)
        return isnothing(idx) ? length(suffixes) + 1 : idx  # Default to placing non-matching names at the end
    end

        # Resort column order based on suffix
    sorted_cols = sort(names(results_8760_df), by=suffix_sort_key) # Sort column names based on suffix priority
    results_8760_df = results_8760_df[:, sorted_cols] # Reorder column names
    
    Macro.write_csv(joinpath(results_dir, case * "_8760_results.csv"), results_8760_df)

    @save case * "_results.jld2" system_results_df capacity_results_df results_8760_df
    println("wrote results")
end

In [16]:
# Run single case
(system, model) = run_case(@__DIR__; optimizer=Gurobi.Optimizer);

###### ###### ######
Running case at /home/al3792/Macro/ExampleSystems/three_zones_macro_genx_cement


┌ Info: Loading system from /home/al3792/Macro/ExampleSystems/three_zones_macro_genx_cement/system_data.json
└ @ MacroEnergy /home/al3792/Macro/src/load_inputs/load_system.jl:18
┌ Info: Loading system data
└ @ MacroEnergy /home/al3792/Macro/src/load_inputs/load_system_data.jl:18
┌ Info: Done loading system data. It took 0.0 seconds
└ @ MacroEnergy /home/al3792/Macro/src/load_inputs/load_system_data.jl:26
┌ Info: Generating system
└ @ MacroEnergy /home/al3792/Macro/src/load_inputs/generate_system.jl:18
┌ Warning: WeightTotal not found in time_data.json
└ @ MacroEnergy /home/al3792/Macro/src/load_inputs/load_time_data.jl:81
┌ Info: Using PeriodLength as default value for WeightTotal
└ @ MacroEnergy /home/al3792/Macro/src/load_inputs/load_time_data.jl:82
┌ Info: Done generating system. It took 0.88 seconds
└ @ MacroEnergy /home/al3792/Macro/src/load_inputs/generate_system.jl:35
┌ Info: Done loading system. It took 0.88 seconds
└ @ MacroEnergy /home/al3792/Macro/src/load_inputs/load_system

Set parameter LicenseID to value 197246
Set parameter GURO_PAR_SPECIAL
Set parameter TokenServer to value "license.rc.princeton.edu"
Set parameter BarConvTol to value 0.001
Set parameter Crossover to value 0
Set parameter Method to value 2
Set parameter Method to value 2
Set parameter Crossover to value 0
Set parameter BarConvTol to value 0.001
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Red Hat Enterprise Linux 8.10 (Ootpa)")

CPU model: Intel(R) Xeon(R) Gold 6246R CPU @ 3.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 32 physical cores, 32 logical processors, using up to 32 threads

Non-default parameters:
Method  2
BarConvTol  0.001
Crossover  0

Optimize a model with 998668 rows, 683330 columns and 3371558 nonzeros
Model fingerprint: 0xff563a51
Coefficient statistics:
  Matrix range     [1e-03, 2e+04]
  Objective range  [1e-01, 2e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+04]
Presolve removed 473603 rows and 272125 columns
Pre

┌ Info: Scaling constraints and RHS
└ @ MacroEnergy /home/al3792/Macro/src/utilities/run_tools.jl:26
┌ Info: Writing capacity results to /home/al3792/Macro/ExampleSystems/three_zones_macro_genx_cement/results/capacity.csv
└ @ MacroEnergy /home/al3792/Macro/src/write_outputs/capacity.jl:118
┌ Info: Writing costs to /home/al3792/Macro/ExampleSystems/three_zones_macro_genx_cement/results/costs.csv
└ @ MacroEnergy /home/al3792/Macro/src/write_outputs/costs.jl:45


In [ ]:
# Load system
case_path = @__DIR__
println("###### ###### ######")
println("Running case at $(case_path)")
system = Macro.load_system(case_path)

###### ###### ######
Running case at /home/al3792/Macro/ExampleSystems/three_zones_macro_genx_cement


UndefVarError: UndefVarError: `Macro` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [13]:
model = Macro.generate_model(system)
Macro.set_optimizer(model, Gurobi.Optimizer);
Macro.set_optimizer_attributes(model, "BarConvTol"=>1e-3,"Crossover" => 0, "Method" => 2)
Macro.optimize!(model)

┌ Info: Starting model generation
└ @ Macro /home/al3792/Macro/src/generate_model.jl:3
┌ Info: Adding linking variables
└ @ Macro /home/al3792/Macro/src/generate_model.jl:17
┌ Info: Defining available capacity
└ @ Macro /home/al3792/Macro/src/generate_model.jl:20
┌ Info: Generating planning model
└ @ Macro /home/al3792/Macro/src/generate_model.jl:23
┌ Info: Generating operational model
└ @ Macro /home/al3792/Macro/src/generate_model.jl:26
┌ Info: Model generation complete, it took 15.816049098968506 seconds
└ @ Macro /home/al3792/Macro/src/generate_model.jl:31


Set parameter LicenseID to value 197246
Set parameter GURO_PAR_SPECIAL
Set parameter TokenServer to value "license.rc.princeton.edu"
Set parameter BarConvTol to value 0.001
Set parameter Crossover to value 0
Set parameter Method to value 2
Set parameter Method to value 2
Set parameter Crossover to value 0
Set parameter BarConvTol to value 0.001
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Red Hat Enterprise Linux 8.10 (Ootpa)")

CPU model: Intel(R) Xeon(R) Gold 6246R CPU @ 3.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 32 physical cores, 32 logical processors, using up to 32 threads

Optimize a model with 998668 rows, 683330 columns and 3371558 nonzeros
Model fingerprint: 0x7ef1d1b6
Coefficient statistics:
  Matrix range     [4e-07, 2e+04]
  Objective range  [1e-01, 2e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+04]
Presolve removed 377243 rows and 272122 columns
Presolve time: 2.80s
Presolved: 621425 rows, 411208 columns, 2442432 

In [26]:
capacity_results_df = Macro.get_optimal_asset_capacity(system)

Row,asset,type,capacity,additions,retirements
,Symbol,Symbol,Float64,Float64,Float64
1,cement_MA,Macro.CementPlant{NaturalGas},72.0477,0.631167,0.0
2,cement_CT,Macro.CementPlant{NaturalGas},124.235,1.08835,0.0
3,cement_ME,Macro.CementPlant{NaturalGas},70.2394,0.615325,0.0
4,sorbent_dac_MA,ElectricDAC,0.0885983,0.0885983,0.0
5,sorbent_dac_CT,ElectricDAC,0.0907732,0.0907732,0.0
6,sorbent_dac_ME,ElectricDAC,0.094898,0.094898,0.0
7,cement_MA,Macro.ElectrochemCementPlant{CementMaterials},7.32549,0.0641742,0.0
8,cement_CT,Macro.ElectrochemCementPlant{CementMaterials},16.4284,0.143919,0.0
9,cement_ME,Macro.ElectrochemCementPlant{CementMaterials},243.697,2.13489,0.0


In [27]:
results_8760_df = DataFrame()
# Get flows of each asset
for i in eachindex(system.assets)
get_flows(system.assets[i], results_8760_df)
end

# Function to get sorting key: returns the index of the first matching suffix or a large number
suffixes = ["cement", "co2"] # Define suffixes to reorder dataframe by
function suffix_sort_key(name)
idx = findfirst(s -> endswith(name, s), suffixes)
return isnothing(idx) ? length(suffixes) + 1 : idx  # Default to placing non-matching names at the end
end

# Resort column order based on suffix
sorted_cols = sort(names(results_8760_df), by=suffix_sort_key) # Sort column names based on suffix priority
results_8760_df = results_8760_df[:, sorted_cols] # Reorder column names

Row,trad_cement_MA_cement,trad_cement_CT_cement,trad_cement_ME_cement,echem_cement_MA_cement,echem_cement_CT_cement,echem_cement_ME_cement,trad_cement_MA_co2,trad_cement_CT_co2,trad_cement_ME_co2,sorbent_dac_MA_co2,sorbent_dac_CT_co2,sorbent_dac_ME_co2,ng_MA_co2,ng_CT_co2,ng_ME_co2,MA_to_CT,MA_to_ME,storage_MA_charge,storage_MA_discharge,storage_CT_charge,storage_CT_discharge,storage_ME_charge,storage_ME_discharge,ng_MA,ng_CT,ng_ME,solar_pv_MA,solar_pv_CT,onshore_wind_CT,onshore_wind_ME
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0268094,0.0264419,0.026965,0.0281324,0.0265053,151.509,0.0208041,0.0205189,0.0209248,0.0436985,0.0466771,0.0660064,1005.96,308.184,0.326848,-292.609,-2000.66,-0.728086,1.42653,-0.702217,1.3575,-7.29285,0.759571,5556.29,1702.19,1.77612,0.0,0.0,832.026,3393.88
2,0.0245387,0.0239749,0.0245747,0.0217612,0.0250103,151.524,0.019042,0.0186045,0.0190699,0.0446397,0.0478854,0.0677226,892.279,308.401,0.3225,-493.973,-2000.86,-0.67249,1.69761,-0.665529,1.65041,-6.72923,0.748332,4928.38,1703.39,1.74721,0.0,0.0,909.864,3335.59
3,0.0267999,0.0263722,0.0267138,0.0329768,0.0384324,151.493,0.0207967,0.0204648,0.0207299,0.0498883,0.050904,0.0668961,807.768,302.452,0.326805,-650.054,-2000.51,-5.28684,0.416374,-4.97277,0.408272,-6.59568,0.914933,4461.6,1670.53,1.76346,0.0,0.0,1013.4,3291.86
4,0.0284128,0.0279763,0.0279398,1.63101,3.27605,146.652,0.0220484,0.0217096,0.0216813,0.0589579,0.0618021,0.0654583,807.657,302.391,0.332831,-604.33,-1893.08,-7.78896,0.0831755,-7.52631,0.0817335,-6.89204,0.826852,4460.98,1670.19,1.78678,0.0,0.0,932.734,3152.62
5,0.0445513,0.0435968,0.0436172,1.20706,2.24798,148.057,0.0345718,0.0338311,0.033847,0.0463261,0.0495955,0.0521561,857.217,308.096,0.448966,-278.115,-1910.87,-0.666929,1.70864,-0.634776,1.68749,-0.606215,3.32446,4734.72,1701.7,2.41495,0.0,0.0,557.307,3160.89
6,0.0431172,0.0421827,0.0422943,1.05128,2.26501,148.2,0.0334589,0.0327338,0.0328204,0.044282,0.047273,0.0509327,963.062,307.926,0.453042,-41.566,-1685.81,-0.714109,1.411,-0.654266,1.42264,-0.68353,1.79883,5319.34,1700.76,2.44306,0.0,0.0,357.009,2953.69
7,0.0430337,0.0421437,0.0422152,1.02914,2.26869,148.219,0.0333941,0.0327035,0.032759,0.0436122,0.0467956,0.0507162,1067.28,307.811,0.456366,-36.9901,-1376.87,-0.753563,1.27653,-0.687688,1.25093,-0.700153,1.54881,5894.97,1700.13,2.46522,0.0,0.0,428.274,2681.04
8,0.0429211,0.042084,0.0421072,1.04729,2.3221,148.147,0.0333068,0.0326572,0.0326752,0.0430866,0.0465563,0.0503483,1145.07,307.76,0.462905,-93.8333,-1127.52,-0.791403,1.1951,-0.736051,1.13019,-0.68901,1.43261,6324.64,1699.84,2.5045,0.0,0.0,552.682,2464.6
9,0.0444013,0.0439331,0.0444845,1.03927,2.44242,148.029,0.0344554,0.0340921,0.03452,0.0428014,0.0459499,0.0496404,1344.92,308.03,0.466751,91.6945,-611.076,-0.801647,1.1185,-0.768613,1.07092,-0.669517,1.3704,7428.51,1701.33,2.52889,0.168109,0.250339,480.754,2001.93


In [33]:
@save "dataframes.jld2" capacity_results_df results_8760_df

In [31]:
@load "dataframes.jld2" capacity_results_df results_8760_df

2-element Vector{Symbol}:
 :capacity_results_df
 :results_8760_df

In [53]:
co2_node = Macro.get_nodes_sametype(system.locations, CO2)[1]
co2_captured_node = Macro.get_nodes_sametype(system.locations, CO2Captured)[1]

system_results_df = DataFrame(
    objective_value = Macro.objective_value(model),
    co2_emissions = Macro.value(sum(co2_captured_node.operation_expr[:exogenous])),
    co2_captured = Macro.value(sum(co2_node.operation_expr[:emissions])),
)

Row,objective_value,co2_emissions,co2_captured
,Float64,Float64,Float64
1,3.85477e9,854.782,1.36619e7


In [47]:
capacity_results = Macro.get_optimal_asset_capacity(system)

Row,asset,type,capacity,additions,retirements
,Symbol,Symbol,Float64,Float64,Float64
1,cement_MA,Macro.CementPlant{NaturalGas},80.2639,0.703144,0.0
2,cement_CT,Macro.CementPlant{NaturalGas},105.652,0.925554,0.0
3,cement_ME,Macro.CementPlant{NaturalGas},80.5629,0.705764,0.0
4,sorbent_dac_MA,ElectricDAC,0.0435383,0.0435383,0.0
5,sorbent_dac_CT,ElectricDAC,0.048591,0.048591,0.0
6,sorbent_dac_ME,ElectricDAC,0.0629949,0.0629949,0.0
7,cement_MA,Macro.ElectrochemCementPlant{CementMaterials},1.61686,0.0141643,0.0
8,cement_CT,Macro.ElectrochemCementPlant{CementMaterials},62.3354,0.546083,0.0
9,cement_ME,Macro.ElectrochemCementPlant{CementMaterials},244.391,2.14096,0.0


In [4]:
# case_path = @__DIR__
# println("###### ###### ######")
# println("Running case at $(case_path)")

# system = Macro.load_system(case_path)

# model = Macro.generate_model(system)

# Macro.set_optimizer(model, Gurobi.Optimizer);

# Macro.set_optimizer_attributes(model, "BarConvTol"=>1e-3,"Crossover" => 0, "Method" => 2)

# Macro.optimize!(model)

###### ###### ######
Running case at /home/al3792/Macro/ExampleSystems/three_zones_macro_genx_cement


┌ Info: Loading JSON data from /home/al3792/Macro/ExampleSystems/three_zones_macro_genx_cement/system_data.json
└ @ Macro /home/al3792/Macro/src/load_inputs/load_macroobject.jl:135
┌ Info: Loading JSON data from /home/al3792/Macro/ExampleSystems/three_zones_macro_genx_cement/system_data.json
└ @ Macro /home/al3792/Macro/src/load_inputs/load_macroobject.jl:135
┌ Info: Loading JSON data from system/nodes.json
└ @ Macro /home/al3792/Macro/src/load_inputs/load_macroobject.jl:135
┌ Info: Loading CSV data from system/demand.csv
└ @ Macro /home/al3792/Macro/src/load_inputs/file_io/csv.jl:12
┌ Info: Loading CSV data from system/demand.csv
└ @ Macro /home/al3792/Macro/src/load_inputs/file_io/csv.jl:14
┌ Info: Loading CSV data from system/hourly_cement_demand.csv
└ @ Macro /home/al3792/Macro/src/load_inputs/file_io/csv.jl:12
┌ Info: Loading CSV data from system/hourly_cement_demand.csv
└ @ Macro /home/al3792/Macro/src/load_inputs/file_io/csv.jl:14
┌ Info: Loading JSON data from assets/cementplan

system.settings.Scaling = false


Macro.System("/home/al3792/Macro/ExampleSystems/three_zones_macro_genx_cement", (Scaling = false,), Dict{Symbol, DataType}(:Electricity => Electricity, :Cement => Cement, :NaturalGas => NaturalGas, :CO2 => CO2, :CementMaterials => CementMaterials, :CO2Captured => CO2Captured), Dict{Symbol, Macro.TimeData}(:Cement => Macro.TimeData{Cement}(1:1:8760, 1, StepRange{Int64, Int64}[1:1:8760], [1], Dict(1 => 1.0)), :NaturalGas => Macro.TimeData{NaturalGas}(1:1:8760, 1, StepRange{Int64, Int64}[1:1:8760], [1], Dict(1 => 1.0)), :CO2 => Macro.TimeData{CO2}(1:1:8760, 1, StepRange{Int64, Int64}[1:1:8760], [1], Dict(1 => 1.0)), :Electricity => Macro.TimeData{Electricity}(1:1:8760, 1, StepRange{Int64, Int64}[1:1:8760], [1], Dict(1 => 1.0)), :CementMaterials => Macro.TimeData{CementMaterials}(1:1:8760, 1, StepRange{Int64, Int64}[1:1:8760], [1], Dict(1 => 1.0)), :CO2Captured => Macro.TimeData{CO2Captured}(1:1:8760, 1, StepRange{Int64, Int64}[1:1:8760], [1], Dict(1 => 1.0))), AbstractAsset[Macro.CementPl

In [85]:
case = "100pct_co2_reduction"

capacity_results = Macro.get_optimal_asset_capacity(system)
results_dir = joinpath(case_path, "results")
mkpath(results_dir)
Macro.write_csv(joinpath(results_dir, case * "_capacity.csv"), capacity_results)
println("wrote results")

wrote results


In [38]:
Macro.type(system.assets[1])

UndefVarError: UndefVarError: `type` not defined in `Macro`
Suggestion: check for spelling errors or missing imports.

In [6]:
co2_nodes = Macro.get_nodes_sametype(system.locations, CO2)
#Macro.value(sum(co2_nodes[1].operation_expr[:emissions]))

1-element Vector{Node}:
 Node{CO2}(:co2_sink, Macro.TimeData{CO2}(1:1:8760, 1, StepRange{Int64, Int64}[1:1:8760], [1], Dict(1 => 1.0)), Dict(:emissions => Dict()), AbstractTypeConstraint[CO2CapConstraint(missing, missing, missing)], Dict{Any, Any}(), Float64[], [0.0], [0.0], Matrix{VariableRef}(undef, 0, 0), Dict{Any, Any}(), Dict{Any, Any}(), Float64[], [0.0], [0.0], Dict{DataType, Float64}(), Dict{DataType, Float64}(CO2CapConstraint => 0.0), Matrix{VariableRef}(undef, 0, 0))

In [81]:
co2_captured_nodes = Macro.get_nodes_sametype(system.locations, CO2Captured)
Macro.value(sum(co2_captured_nodes[1].operation_expr[:exogenous]))

4.087119470692836e6

In [16]:
co2_nodes[1].rhs_policy[CO2CapConstraint] = 1

1

In [17]:
co2_nodes[1].rhs_policy[CO2CapConstraint]

1.0

In [8]:
Macro.value.(co2_nodes[1].policy_budgeting_vars[:CO2CapConstraint_Budget])

KeyError: KeyError: key :CO2CapConstraint_Budget not found

In [35]:
Macro.investment_cost(system.assets[1].cement_edge)

187.0

In [36]:
Macro.capacity_size(system.assets[1].cement_edge)

1.0

In [37]:
model[:eFixedCost]

208.45 vNEWCAP_cement_MA_cement_edge - 21.45 vRETCAP_cement_MA_cement_edge + 208.45 vNEWCAP_cement_CT_cement_edge - 21.45 vRETCAP_cement_CT_cement_edge + 208.45 vNEWCAP_cement_ME_cement_edge - 21.45 vRETCAP_cement_ME_cement_edge + 1686000 vNEWCAP_sorbent_dac_MA_co2_edge - 747000 vRETCAP_sorbent_dac_MA_co2_edge + 1686000 vNEWCAP_sorbent_dac_CT_co2_edge - 747000 vRETCAP_sorbent_dac_CT_co2_edge + 1686000 vNEWCAP_sorbent_dac_ME_co2_edge - 747000 vRETCAP_sorbent_dac_ME_co2_edge + 301.95 vNEWCAP_cement_MA_cement_edge - 21.45 vRETCAP_cement_MA_cement_edge + 301.95 vNEWCAP_cement_CT_cement_edge - 21.45 vRETCAP_cement_CT_cement_edge + 301.95 vNEWCAP_cement_ME_cement_edge - 21.45 vRETCAP_cement_ME_cement_edge + 12060 vNEWCAP_MA_to_CT_elec_edge + 19261 vNEWCAP_MA_to_ME_elec_edge + 28116 vNEWCAPSTOR_storage_MA_storage - 5622 vRETCAPSTOR_storage_MA_storage + 24479 vNEWCAP_storage_MA_discharge_edge - 4895 vRETCAP_storage_MA_discharge_edge + 28116 vNEWCAPSTOR_storage_CT_storage - 5622 vRETCAPSTOR_sto

In [8]:
co2_captured_nodes = Macro.get_nodes_sametype(system.locations, CO2Captured)

1-element Vector{Node}:
 Node{CO2Captured}(:co2_captured_sink, Macro.TimeData{CO2Captured}(1:1:8760, 1, StepRange{Int64, Int64}[1:1:8760], [1], Dict(1 => 1.0)), Dict(:exogenous => Dict()), AbstractTypeConstraint[], Dict{Any, Any}(:exogenous => 1-dimensional DenseAxisArray{AffExpr,1,...} with index sets:
    Dimension 1, 1:1:8760
And data, a 8760-element Vector{AffExpr}:
 vFLOW_sorbent_dac_MA_co2_captured_edge[1] + vFLOW_sorbent_dac_CT_co2_captured_edge[1] + vFLOW_sorbent_dac_ME_co2_captured_edge[1]
 vFLOW_sorbent_dac_MA_co2_captured_edge[2] + vFLOW_sorbent_dac_CT_co2_captured_edge[2] + vFLOW_sorbent_dac_ME_co2_captured_edge[2]
 vFLOW_sorbent_dac_MA_co2_captured_edge[3] + vFLOW_sorbent_dac_CT_co2_captured_edge[3] + vFLOW_sorbent_dac_ME_co2_captured_edge[3]
 vFLOW_sorbent_dac_MA_co2_captured_edge[4] + vFLOW_sorbent_dac_CT_co2_captured_edge[4] + vFLOW_sorbent_dac_ME_co2_captured_edge[4]
 vFLOW_sorbent_dac_MA_co2_captured_edge[5] + vFLOW_sorbent_dac_CT_co2_captured_edge[5] + vFLOW_sorbent_

In [9]:
co2_node.operation_expr[:emissions][1]

vFLOW_cement_MA_co2_edge[1] + vFLOW_cement_CT_co2_edge[1] + vFLOW_cement_ME_co2_edge[1] - vFLOW_sorbent_dac_MA_co2_edge[1] - vFLOW_sorbent_dac_CT_co2_edge[1] - vFLOW_sorbent_dac_ME_co2_edge[1] + vFLOW_ng_MA_co2_edge[1] + vFLOW_ng_CT_co2_edge[1] + vFLOW_ng_ME_co2_edge[1]

In [10]:
co2_node.constraints[1].constraint_ref.data[1]

-vCO2CapConstraint_Budget_co2_sink[1] + vFLOW_cement_MA_co2_edge[1] + vFLOW_cement_MA_co2_edge[2] + vFLOW_cement_MA_co2_edge[3] + vFLOW_cement_MA_co2_edge[4] + vFLOW_cement_MA_co2_edge[5] + vFLOW_cement_MA_co2_edge[6] + vFLOW_cement_MA_co2_edge[7] + vFLOW_cement_MA_co2_edge[8] + vFLOW_cement_MA_co2_edge[9] + vFLOW_cement_MA_co2_edge[10] + vFLOW_cement_MA_co2_edge[11] + vFLOW_cement_MA_co2_edge[12] + vFLOW_cement_MA_co2_edge[13] + vFLOW_cement_MA_co2_edge[14] + vFLOW_cement_MA_co2_edge[15] + vFLOW_cement_MA_co2_edge[16] + vFLOW_cement_MA_co2_edge[17] + vFLOW_cement_MA_co2_edge[18] + vFLOW_cement_MA_co2_edge[19] + vFLOW_cement_MA_co2_edge[20] + vFLOW_cement_MA_co2_edge[21] + vFLOW_cement_MA_co2_edge[22] + vFLOW_cement_MA_co2_edge[23] + vFLOW_cement_MA_co2_edge[24] + vFLOW_cement_MA_co2_edge[25] + vFLOW_cement_MA_co2_edge[26] + vFLOW_cement_MA_co2_edge[27] + vFLOW_cement_MA_co2_edge[28] + vFLOW_cement_MA_co2_edge[29] + [[...78781 terms omitted...]] + vFLOW_ng_ME_co2_edge[8731] + vFLOW_ng_

In [33]:
system.assets[11].elec_edge

Edge{Electricity}(:MA_to_ME_elec_edge, Macro.TimeData{Electricity}(1:1:8760, 1, StepRange{Int64, Int64}[1:1:8760], [1], Dict(1 => 1.0)), Node{Electricity}(:elec_MA, Macro.TimeData{Electricity}(1:1:8760, 1, StepRange{Int64, Int64}[1:1:8760], [1], Dict(1 => 1.0)), Dict(:demand => Dict()), AbstractTypeConstraint[BalanceConstraint(missing, missing, 2-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},2,...} with index sets:
    Dimension 1, [:demand]
    Dimension 2, 1:1:8760
And data, a 1×8760 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 -7850 vREF + vNSD_elec_MA[1,1] + vNSD_elec_MA[2,1] + vNSD_elec_MA[3,1] + vNSD_elec_MA[4,1] - vFLOW_cement_MA_elec_edge[1] - vFLOW_sorbent_dac_MA_elec_edge[1] - vFLOW_cement_MA_elec_edge[1] - vFLOW_MA_to_CT_elec_edge[

In [43]:
Macro.value.(Macro.flow(system.assets[14].storage)).data

ErrorException: type Battery has no field storage

In [52]:
system.assets[18].edge

Edge{Electricity}(:solar_pv_MA_edge, Macro.TimeData{Electricity}(1:1:8760, 1, StepRange{Int64, Int64}[1:1:8760], [1], Dict(1 => 1.0)), Transformation(:solar_pv_MA_transforms, Macro.TimeData{Electricity}(1:1:8760, 1, StepRange{Int64, Int64}[1:1:8760], [1], Dict(1 => 1.0)), Dict{Symbol, Dict{Symbol, Float64}}(), AbstractTypeConstraint[], Dict{Any, Any}()), Node{Electricity}(:elec_MA, Macro.TimeData{Electricity}(1:1:8760, 1, StepRange{Int64, Int64}[1:1:8760], [1], Dict(1 => 1.0)), Dict(:demand => Dict()), AbstractTypeConstraint[BalanceConstraint(missing, missing, 2-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},2,...} with index sets:
    Dimension 1, [:demand]
    Dimension 2, 1:1:8760
And data, a 1×8760 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}

In [40]:
Macro.value.(Macro.flow(system.assets[15].elec_edge)).data

8760-element Vector{Float64}:
  9083.281875876088
  8512.217346006011
  8090.070551938082
  7863.3430963485625
  7832.9118572064
  8008.519062332811
  8378.686384578617
  8708.101905225627
  9269.955119399869
  9639.035686832633
     ⋮
 12361.395694797993
 12738.655659466984
 12912.594716518175
 12830.311308371603
 12372.797145380964
 11933.873273996458
 11364.009217206427
 10738.704728408413
 10141.710377909187

In [86]:
results_8760 = DataFrame(
    # Cement production
    trad_cement_1_produced = Macro.value.(Macro.flow(system.assets[1].cement_edge)).data,
    trad_cement_2_produced = Macro.value.(Macro.flow(system.assets[2].cement_edge)).data,
    trad_cement_3_produced = Macro.value.(Macro.flow(system.assets[3].cement_edge)).data,

    EC_cement_1_produced = Macro.value.(Macro.flow(system.assets[7].cement_edge)).data,
    EC_cement_2_produced = Macro.value.(Macro.flow(system.assets[8].cement_edge)).data,
    EC_cement_3_produced = Macro.value.(Macro.flow(system.assets[9].cement_edge)).data,

    # CO2 emissions
    trad_cement_1_co2 = Macro.value.(Macro.flow(system.assets[1].co2_edge)).data,
    trad_cement_2_co2 = Macro.value.(Macro.flow(system.assets[2].co2_edge)).data,
    trad_cement_3_co2 = Macro.value.(Macro.flow(system.assets[3].co2_edge)).data,

    dac_1_co2 = -1 * Macro.value.(Macro.flow(system.assets[4].co2_captured_edge)).data,
    dac_2_co2 = -1 * Macro.value.(Macro.flow(system.assets[5].co2_captured_edge)).data,
    dac_3_co2 = -1 * Macro.value.(Macro.flow(system.assets[6].co2_captured_edge)).data,

    ng_1_co2 = Macro.value.(Macro.flow(system.assets[15].co2_edge)).data,
    ng_2_co2 = Macro.value.(Macro.flow(system.assets[16].co2_edge)).data,
    ng_3_co2 = Macro.value.(Macro.flow(system.assets[17].co2_edge)).data,

    # Electricity
    MA_to_CT_elec = Macro.value.(Macro.flow(system.assets[10].elec_edge)).data,
    MA_to_ME_elec = Macro.value.(Macro.flow(system.assets[11].elec_edge)).data,

    storage_MA_charge = -1 * Macro.value.(Macro.flow(system.assets[12].charge_edge)).data,
    storage_CT_charge = -1 * Macro.value.(Macro.flow(system.assets[13].charge_edge)).data,
    storage_ME_charge = -1 * Macro.value.(Macro.flow(system.assets[14].charge_edge)).data,

    storage_MA_discharge = Macro.value.(Macro.flow(system.assets[12].discharge_edge)).data,
    storage_CT_discharge = Macro.value.(Macro.flow(system.assets[13].discharge_edge)).data,
    storage_ME_discharge = Macro.value.(Macro.flow(system.assets[14].discharge_edge)).data,

    ng_MA = Macro.value.(Macro.flow(system.assets[15].elec_edge)).data,
    ng_CT = Macro.value.(Macro.flow(system.assets[16].elec_edge)).data,
    ng_ME = Macro.value.(Macro.flow(system.assets[17].elec_edge)).data,

    solar_pv_MA = Macro.value.(Macro.flow(system.assets[18].edge)).data,
    solar_pv_CT = Macro.value.(Macro.flow(system.assets[19].edge)).data,

    onshore_wind_CT = Macro.value.(Macro.flow(system.assets[20].edge)).data,
    onshore_wind_ME = Macro.value.(Macro.flow(system.assets[21].edge)).data,
)

Row,trad_cement_1_produced,trad_cement_2_produced,trad_cement_3_produced,EC_cement_1_produced,EC_cement_2_produced,EC_cement_3_produced,trad_cement_1_co2,trad_cement_2_co2,trad_cement_3_co2,dac_1_co2,dac_2_co2,dac_3_co2,ng_1_co2,ng_2_co2,ng_3_co2,MA_to_CT_elec,MA_to_ME_elec,storage_MA_charge,storage_CT_charge,storage_ME_charge,storage_MA_discharge,storage_CT_discharge,storage_ME_discharge,ng_MA,ng_CT,ng_ME,solar_pv_MA,solar_pv_CT,onshore_wind_CT,onshore_wind_ME
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0148384,0.0146555,0.0150737,0.0297483,0.0412907,151.528,0.0115146,0.0113726,0.0116972,-0.528773,-0.957904,-1149.63,26.859,143.161,0.193313,-4183.74,-3999.82,-480.361,-367.908,-62.743,0.860237,0.837874,20.7274,148.319,790.696,1.01292,0.0,0.0,6006.4,10464.4
2,0.0152317,0.0150707,0.0154494,0.0277302,0.0391775,151.531,0.0118198,0.0116949,0.0119887,-0.528434,-0.959315,-1149.63,26.8458,143.149,0.193327,-4473.93,-3999.82,-1196.56,-761.57,-61.0701,0.942904,0.907366,21.0605,148.238,790.624,1.00744,0.0,0.0,6568.25,10404.4
3,0.0151237,0.0149864,0.0153496,0.0422398,0.0644178,151.492,0.011736,0.0116294,0.0119113,-0.528585,-0.966782,-1149.63,26.8407,143.142,0.195713,-4678.69,-3999.73,-1718.07,-1395.03,-61.7351,0.861805,0.866758,20.8631,148.198,790.579,1.01351,0.0,0.0,7315.65,10362.1
4,0.0152013,0.0150343,0.0154445,0.0228136,0.0315991,151.544,0.0117962,0.0116666,0.0119849,-0.527359,-0.955143,-1149.63,26.8453,143.141,0.200504,-4382.84,-3999.86,-1582.92,-1243.55,-65.1074,1.37417,1.38154,20.1768,148.205,790.564,1.03089,0.0,0.0,6822.7,10344.4
5,0.0146674,0.014681,0.0147701,0.0396831,0.0422953,151.518,0.0113819,0.0113925,0.0114616,-0.539014,-0.964636,-1149.63,26.8445,143.128,0.208118,-2804.79,-3999.78,-29.289,-28.7304,-70.2258,0.993858,1.05818,19.5375,148.165,790.482,1.06132,0.0,0.0,4023.3,10347.0
6,0.014388,0.0143919,0.0147576,0.0170366,0.0181508,151.565,0.0111651,0.0111681,0.0114519,-0.463318,-0.901089,-1149.63,164.411,342.018,0.21946,-2451.96,-3999.9,-0.671699,-0.770848,-73.2037,2.31819,2.17021,19.7995,593.554,1888.99,1.10712,0.0,0.0,2577.56,10365.9
7,0.0144706,0.0144757,0.0148463,0.0172428,0.0182237,151.565,0.0112292,0.0112332,0.0115207,-0.458927,-0.893184,-1149.63,134.274,289.186,0.238356,-2598.52,-3999.9,-0.842955,-0.883303,-80.4368,1.20826,1.17968,19.2887,710.26,1597.18,1.18067,0.0,0.0,3092.0,10409.5
8,0.0146778,0.0146817,0.0150527,0.0176566,0.0189341,151.563,0.01139,0.011393,0.0116809,-0.457866,-0.869729,-1149.63,122.755,191.231,0.276489,-2888.27,-3999.9,-0.967161,-0.949158,-120.24,0.965821,0.960146,15.1019,657.871,1056.13,1.31897,0.0,0.0,3989.98,10486.4
9,0.0267907,0.0268159,0.026938,0.911144,75.8084,74.8437,0.0207896,0.0208091,0.0209039,-0.489969,-0.87871,-1149.51,112.713,183.617,15.8822,-2988.88,-2646.12,-1.43963,-1.12015,-0.690738,0.629925,0.804897,1.47323,605.942,1014.07,44.2093,1709.94,936.385,3470.78,8876.14


In [83]:
Macro.write_csv(joinpath(results_dir, case * "_8760_results.csv"), results_8760)

"/home/al3792/Macro/ExampleSystems/three_zones_macro_genx_cement/results/80pct_co2_reduction_8760_results.csv"

In [15]:
maximum.(eachcol(results_8760))

6-element Vector{Float64}:
  85.35149561375196
  85.34579048819228
  95.95184464105915
   7.025230945671713
 244.66559703505843
 247.2520577653591

In [23]:
sum(eachcol(results_8760))

8760-element Vector{Float64}:
 151.6438172060847
 151.64381720695397
 151.64381720695397
 151.64381720695397
 151.64381720608472
 151.64381720608472
 151.64381720608472
 151.64381720608472
 151.64381720608475
 151.64381720456353
   ⋮
 208.94220430608468
 208.94220430608473
 208.94220430695393
 208.94220430695395
 208.9422043060847
 208.94220430608445
 208.94220430608468
 208.94220430695393
 208.94220430695393

In [30]:
co2_results_8760 = DataFrame(
    cement_1_co2 = Macro.value.(Macro.flow(system.assets[1].co2_edge)).data,
    cement_2_co2 = Macro.value.(Macro.flow(system.assets[2].co2_edge)).data,
    cement_3_co2 = Macro.value.(Macro.flow(system.assets[3].co2_edge)).data,

    dac_1_co2 = -1 * Macro.value.(Macro.flow(system.assets[4].co2_captured_edge)).data,
    dac_2_co2 = -1 * Macro.value.(Macro.flow(system.assets[5].co2_captured_edge)).data,
    dac_3_co2 = -1 * Macro.value.(Macro.flow(system.assets[6].co2_captured_edge)).data,

    ng_1_co2 = Macro.value.(Macro.flow(system.assets[15].co2_edge)).data,
    ng_2_co2 = Macro.value.(Macro.flow(system.assets[16].co2_edge)).data,
    ng_3_co2 = Macro.value.(Macro.flow(system.assets[17].co2_edge)).data
)

Row,cement_1_co2,cement_2_co2,cement_3_co2,dac_1_co2,dac_2_co2,dac_3_co2,ng_1_co2,ng_2_co2,ng_3_co2
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.00971605,0.00963162,0.00987397,-0.47474,-0.90787,-1152.11,44.2065,149.616,0.177597
2,0.00968595,0.00960725,0.00981929,-0.475061,-0.909826,-1152.12,44.1996,149.61,0.177614
3,0.00971419,0.00964494,0.00985212,-0.475083,-0.917829,-1152.12,44.2009,149.609,0.17994
4,0.00968054,0.00959702,0.00982999,-0.473137,-0.906805,-1152.12,44.2139,149.613,0.184434
5,0.00959271,0.00962656,0.00973084,-0.473644,-0.903734,-1152.11,44.2567,149.625,0.191729
6,0.00946875,0.00949387,0.00974292,-0.409303,-0.839792,-1152.11,90.628,375.622,0.202818
7,0.00955658,0.00958422,0.00983205,-0.402744,-0.830359,-1152.11,92.9519,340.043,0.221956
8,0.00984282,0.00986542,0.0101034,-0.399936,-0.818306,-1152.11,93.1127,235.34,0.264023
9,0.01825,0.0183664,0.0184265,-0.431045,-0.821386,-1152.0,82.0077,185.081,68.8892


In [36]:
sum.(eachcol(sum.(eachrow(co2_results_8760))))

1-element Vector{Float64}:
 -40.0001745801419